# Building Semantic Memory with Embeddings

So far, we've mostly been treating the kernel as a stateless orchestration engine.
We send text into a model API and receive text out. 

In a [previous notebook](04-kernel-arguments-chat.ipynb), we used `kernel arguments` to pass in additional
text into prompts to enrich them with more data. This allowed us to create a basic chat experience. 

However, if you solely relied on kernel arguments, you would quickly realize that eventually your prompt
would grow so large that you would run into a the model's token limit. What we need is a way to persist state
and build both short-term and long-term memory to empower even more intelligent applications. 

To do this, we dive into the key concept of `Semantic Memory` in the Semantic Kernel. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.2.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.2.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Sqlite, 1.2.0-alpha"
#r "nuget: System.Linq.Async, 6.0.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Connectors.Sqlite, 1.2.0-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.2.0-alpha System.Linq.Async, 6.0.1

In order to use memory, we need to instantiate the Memory Plugin with a Memory Storage
and an Embedding backend. In this example, we make use of the `VolatileMemoryStore`
which can be thought of as a temporary in-memory storage (not to be confused with Semantic Memory).

This memory is not written to disk and is only available during the app session.

When developing your app you will have the option to plug in persistent storage
like Azure Cosmos Db, PostgreSQL, SQLite, etc. Semantic Memory allows also to index
external data sources, without duplicating all the information, more on that later.

In [2]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.Sqlite;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Memory functionality is experimental
#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0052, SKEXP0028

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var index = !File.Exists("index.db");

var store = await SqliteMemoryStore.ConnectAsync("index.db");

var memoryBuilder = new MemoryBuilder();

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
        "text-embedding-ada-002",
        azureEndpoint, 
        apiKey,
        "model-id");
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey);
}


memoryBuilder.WithMemoryStore(store);

var memory = memoryBuilder.Build();

Let's try searching the memory:

In [3]:
const string MemoryCollectionName = "CocktailCollection";


var questions = new[]
{
    "list of good sparking cocktails",
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}

list of good sparking cocktails Russian Spring Punch, Glass: highball, Category: Sparkling Cocktail, Ingredients: 2.5 cl Vodka, 2.5 cl Lemon juice, 1.5 cl Créme liqueur (Créme de Cassis), 1 cl Syrup (Sugar syrup), Garnish: Lemon slice and a blackberry, Preparation: Shake the ingredients and pour into highball glass. Top with Sparkling wine.


In [4]:
using Microsoft.SemanticKernel.Plugins.Memory;

#pragma warning disable SKEXP0052

// TextMemoryPlugin provides the "recall" function
kernel.ImportPluginFromObject(new TextMemoryPlugin(memory));

In [7]:
#pragma warning disable SKEXP0003
var query = await InteractiveKernel.GetInputAsync("What is your query?");

IAsyncEnumerable<MemoryQueryResult> queryResults =
                memory.SearchAsync(MemoryCollectionName, query, limit: 3, minRelevanceScore: 0.77);

In [8]:
#pragma warning disable SKEXP0003
// Keep a list of the memories
StringBuilder promptData = new StringBuilder();

await foreach (MemoryQueryResult r in queryResults)
{
    promptData.Append(r.Metadata.Text+"\n\n");
}

// Final augmented text
var augmentedText = promptData.ToString();
Console.WriteLine($"User:\n{query}\n\nNearest results:\n{augmentedText}")

User:
white rum

Nearest results:
Mai-tai, Glass: highball, Category: Longdrink, Ingredients: 4 cl White rum, 2 cl Dark rum, 1.5 cl Triple Sec (Orange Curaçao), 1.5 cl Syrup (Orgeat syrup), 1 cl Lime juice, Garnish: Pineapple spear, mint leaves and lime wedge, Preparation: Shake and strain into highball glass. Serve with straw.

Bacardi, Glass: martini, Category: Before Dinner Cocktail, Ingredients: 4.5 cl White rum (Bacardi White Rum), 2 cl Lime juice, 1 cl Syrup (Grenadine), Garnish: , Preparation: Shake with ice cubes. Strain into chilled cocktail glass.

Hemingway Special, Glass: martini, Category: All Day Cocktail, Ingredients: 6 cl White rum, 4 cl Grapefruit juice, 1.5 cl Cherry liqueur (Maraschino), 1.5 cl Lime juice, Garnish: , Preparation: Shake with ice cubes. Strain into a double cocktail glass.




In [9]:

const string ragFunctionDefinition = @"
ChatBot can only have a conversation with relevant information below.
It can give explicit instructions or say 'Beats me, I'm just here for the drink' if it does not have an answer.

Relevant information:
{{$data}}

Chat:
{{$history}}
User: {{$userInput}}
ChatBot: ";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.8,
    TopP = 0.5
};


var ragFunction = kernel.CreateFunctionFromPrompt(ragFunctionDefinition, executionSettings);


In [10]:
#pragma warning disable SKEXP0052

var query = "Give me 5 drinks with white rum"; // await InteractiveKernel.GetInputAsync("What is your query?");

var arguments = new KernelArguments();

arguments["userInput"] = query;
arguments[TextMemoryPlugin.CollectionParam] = MemoryCollectionName;
arguments[TextMemoryPlugin.LimitParam] = "3";
arguments[TextMemoryPlugin.RelevanceParam] = "0.8";

var result = await kernel.InvokeAsync(ragFunction, arguments);

Console.WriteLine(result);



Sure! Here are five drinks that you can make with white rum:

1. Mojito: Mix white rum, fresh lime juice, sugar, mint leaves, and soda water for a refreshing cocktail.
2. Piña Colada: Blend white rum, pineapple juice, coconut cream, and ice for a tropical treat.
3. Daiquiri: Shake white rum, lime juice, and simple syrup with ice for a classic cocktail.
4. Cuba Libre: Combine white rum, cola, and a squeeze of lime for a simple and delicious drink.
5. Mai Tai: Mix white rum, orange liqueur, lime juice, almond syrup, and a splash of grenadine for a tropical and fruity cocktail.

Enjoy your drinks!
